In [ ]:
# convert the .png annotations to the txt format of MOTS
# In other words: convert the STEP annotation to the compatible json format used by MOTSDataLoader

# References
# KITTI-STEP annotation format: https://github.com/google-research/deeplab2/blob/main/g3doc/setup/kitti_step.md
# MOTS TXT annotation format: https://www.vision.rwth-aachen.de/page/mots

# download labels from
# http://www.cvlibs.net/datasets/kitti/eval_step.php direct link: https://storage.googleapis.com/gresearch/tf-deeplab/data/kitti-step.tar.gz
# We need the data from sequenze 2 from (moved to sequenze 50) :https://motchallenge.net/data/STEP-ICCV21/ direct link: https://motchallenge.net/data/motchallenge-step.tar.gz


In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pycocotools import mask
from tqdm import tqdm
import json

In [ ]:
ano_dir = Path("/home/rune/thesis/data/step/kitti-step/panoptic_maps")
# TODO: load json for the image paths
example_annotation = Path(ano_dir) / "train" / "0000" / "000000.png"

In [ ]:
def encode_mask(mask_array):
    to_encode = np.asfortranarray(mask_array, dtype=np.uint8)
    encoded = mask.encode(to_encode)["counts"].decode("utf8")
    return encoded

In [ ]:
sub_folder = "train"
kitti_mots_json = f"/home/rune/thesis/data/dataset_jsons/kittimots_{sub_folder}.json"
# load old data
mots_data = json.load(open(kitti_mots_json, "r"))
category_labels = {'0': 'road',
 '1': 'sidewalk',
 '2': 'building',
 '3': 'wall',
 '4': 'fence',
 '5': 'pole',
 '6': 'trafficlight',
 '7': 'trafficsign',
 '8': 'vegetation',
 '9': 'terrain',
 '10': 'sky',
 '11': 'person',
 '12': 'rider',
 '13': 'car',
 '14': 'truck',
 '15': 'bus',
 '16': 'train',
 '17': 'motorcycle',
 '18': 'bicycle',
 '255': 'void'}
mots_data["meta"]["category_labels"] = category_labels
mots_data["meta"]["name"] = f"kitti_step_{sub_folder}"

for sequenze in mots_data["sequences"]:
    id = sequenze["id"]
    # print(id)
    # mapping from the new_instance_id to the category
    # the new_instance_id is the class_id_instance_id
    categories = {}
    # dict of mask for every instance for every timestamp
    segmentations = []

    for filename in tqdm(sequenze["image_paths"]):
        # TODO: check if the image size matches
        # TODO: use same naming as in the other dataloaders (category and instance)
        # laod annotation
        img = Image.open(open(ano_dir / sub_folder / Path(filename).with_suffix(".png"), "rb"))
        img = np.array(img)
        # convert to instance id and label
        category = img[:,:,0]
        instance= img[:,:,1] * 256 + img[:,:,2]

        instanceids = np.unique(instance)

        image_segmentations = {}
        # convert to masks
        for class_id in np.unique(category):
            category_mask = category == class_id
            # handle pedestrians and cars with respect ot their instance_id
            # TODO: with some computational overhead we could remove this and always take this branch.
            if class_id in [11, 13]:
                for id in instanceids:
                    # check if we have a overlap, meaning we have a instance of that class
                    instance_mask = category_mask & (instance == id)
                    if instance_mask.any():
                        # create a new instanceid
                        new_instance_id = f"{class_id}_{id}"
                        categories[new_instance_id] = int(class_id)
                        image_segmentations[new_instance_id] = encode_mask(instance_mask)
            else:
                new_instance_id = f"{class_id}"
                categories[new_instance_id] = int(class_id)
                image_segmentations[new_instance_id] = encode_mask(category_mask)

        segmentations.append(image_segmentations)
    sequenze["categories"] = categories
    sequenze["segmentations"] = segmentations
    assert len(segmentations) == len(sequenze["image_paths"])
    assert len(segmentations) == sequenze["length"]


In [ ]:
with open(f"kittistep_{sub_folder}.json", "w") as f:
    json.dump(mots_data, f)